In [1]:
from sklearn.decomposition import PCA
import numpy as np
import time
import xarray as xr
import dask
from dask_gateway import Gateway
from dask.distributed import Client, progress
import pandas as pd
import time

In [5]:
gateway = Gateway()
# close existing clusters (be careful if you have multiple clusters/servers open!)
open_clusters = gateway.list_clusters()
print(list(open_clusters))
if len(open_clusters)>0:
    for c in open_clusters:
        cluster = gateway.connect(c.name)
        cluster.shutdown()  

client = Client(threads_per_worker=10, n_workers=12)
client

[]


In [7]:
def pca_test(mat):
    pca = PCA(500)
    pca.fit(mat)
    X_pca = pca.transform(mat)
    return X_pca
    
p = np.random.rand(1000,10000) #random input data

Without dask, +- 3min:

In [12]:
%%time
results=[]
for i in range(100):
    result = pca_test(p)
    results.append(result)

CPU times: user 27min 10s, sys: 22min, total: 49min 11s
Wall time: 3min 13s


With dask, +- 1min (I would expect this to be faster?):

In [8]:
lazy_results=[]
for i in range(100):
    lazy_result = dask.delayed(pca_test)(p)
    lazy_results.append(lazy_result)

In [9]:
%time dask.compute(*lazy_results)

/srv/conda/envs/notebook/lib/python3.10/site-packages/distributed/worker.py:3029: UserWarning: Large object of size 76.29 MiB detected in task graph: 
  (array([[0.58963048, 0.15501055, 0.39788635, ...,  ... 0.89136639]]),)
Consider scattering large objects ahead of time
with client.scatter to reduce scheduler burden and 
keep data on workers

    future = client.submit(func, big_data)    # bad

    big_future = client.scatter(big_data)     # good
    future = client.submit(func, big_future)  # good
  warnings.warn(


CPU times: user 14.5 s, sys: 12.1 s, total: 26.6 s
Wall time: 1min 6s


(array([[-8.18095688e-01, -7.87449762e-01, -8.92334002e-01, ...,
         -1.09306659e-01,  6.73454323e-01, -5.28543745e-02],
        [-7.11252391e-01, -1.07090823e+00, -1.19336550e+00, ...,
          6.31248243e-02,  2.11271090e-01, -4.82836387e-04],
        [ 3.57482681e-02,  3.50113961e-01, -1.13828882e+00, ...,
         -2.05903201e-01,  1.30210254e+00,  1.61694179e+00],
        ...,
        [-1.72055025e+00, -1.01610257e+00,  1.13571505e+00, ...,
          4.90057471e-01, -3.66409165e-01,  5.82006282e-01],
        [-1.79800796e+00,  8.47201904e-01, -5.25694151e-02, ...,
          1.10322974e+00,  7.48022607e-01, -8.83980863e-01],
        [-2.32092416e+00, -1.34214183e+00,  3.64860504e-01, ...,
          1.33458882e+00,  3.61527111e-01, -7.82702374e-02]]),
 array([[ 0.83475974,  0.71929201, -0.86555112, ..., -0.3417051 ,
         -1.23633433, -1.25086888],
        [ 0.77804134,  1.05732061, -1.2320541 , ...,  0.29910192,
          1.71995001, -0.80180617],
        [-0.29220629, -0.